In [1]:
import sys
print(sys.prefix)
from pprint import pprint

/home/jinda/anaconda3/envs/cs224n


In [2]:
!git config --global user.email "jindaz.work@outlook.com"
!git config --global user.name "jindaczmatrix"

In [3]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Wed Dec  6 01:41:17 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.12             Driver Version: 535.104.12   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce GTX 1080 Ti     On  | 00000000:06:00.0 Off |                  N/A |
| 34%   47C    P8              17W / 250W |   2861MiB / 11264MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [4]:
!pip install datasets jiwer g2p g2p_en

In [5]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
!pwd
%cd drive/MyDrive
!pwd

/content
/content/drive/MyDrive
/content/drive/MyDrive


In [ ]:
# ensure enough memory present so that training does not stop
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 67.3 gigabytes of available RAM

You are using a high-RAM runtime!


In [6]:
# Import libraries
import torch
from transformers import BartTokenizerFast, BartForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer
from datasets import load_dataset
from sklearn.model_selection import train_test_split
from jiwer import wer
from huggingface_hub import notebook_login

2023-12-06 14:58:14.294433: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-06 14:58:14.340400: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-06 14:58:15.245895: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [ ]:
notebook_login()

# preprocessing using G2P

In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Read the CSV file into a DataFrame
df = pd.read_csv('data/words_noise_data_sample.csv')
df

,word,phonetic_alterntive
0,oak,owk
1,sorrow,saarow
2,kilograms,kihlahgraemz
3,taxi,taeksiy
4,beth,behth
...,...,...
7099,formats,faormaets
7100,about,ahbawt
7101,humid,hhyuwmahd
7102,four,faor


# init a G2P converter

In [8]:
from g2p_en import G2p

g2p = G2p()

df['word_phoneme'] = df['word'].apply(lambda x: " ".join(g2p(x)))
df['phonetic_alternative_phoneme'] = df['phonetic_alterntive'].apply(lambda x: " ".join(g2p(x)))
df

,word,phonetic_alterntive,word_phoneme,phonetic_alternative_phoneme
0,oak,owk,OW1 K,OW1 K
1,sorrow,saarow,S AA1 R OW0,S AA1 R OW0
2,kilograms,kihlahgraemz,K IH1 L AH0 G R AE2 M Z,K IH2 L HH AH0 G R AA1 M AH0 D IY0
3,taxi,taeksiy,T AE1 K S IY0,T EH1 K S IY0 IH0 S
4,beth,behth,B EH1 TH,B EH1 TH
...,...,...,...,...
7099,formats,faormaets,F AO1 R M AE2 T S,F AO1 R M EY2 T S
7100,about,ahbawt,AH0 B AW1 T,AA1 B AA0 T
7101,humid,hhyuwmahd,HH Y UW1 M AH0 D,HH AY1 W M AH0 N D
7102,four,faor,F AO1 R,F AO1 ER0


In [ ]:
df = df.drop(['word', 'phonetic_alterntive'], axis=1)
print(df)

                 word_phoneme        phonetic_alternative_phoneme
0                       OW1 K                               OW1 K
1                 S AA1 R OW0                         S AA1 R OW0
2     K IH1 L AH0 G R AE2 M Z  K IH2 L HH AH0 G R AA1 M AH0 D IY0
3               T AE1 K S IY0                 T EH1 K S IY0 IH0 S
4                    B EH1 TH                            B EH1 TH
...                       ...                                 ...
7099        F AO1 R M AE2 T S                   F AO1 R M EY2 T S
7100              AH0 B AW1 T                         AA1 B AA0 T
7101         HH Y UW1 M AH0 D                  HH AY1 W M AH0 N D
7102                  F AO1 R                           F AO1 ER0
7103                    AH1 S                               AA1 Z

[7104 rows x 2 columns]


## Remove Numbers and Duplicates

In [ ]:
# Drop all numbers in the 'word_phoneme' and 'phonetic_alternative_phoneme' columns
df['word_phoneme'] = df['word_phoneme'].str.replace(r'\d', '')
df['phonetic_alternative_phoneme'] = df['phonetic_alternative_phoneme'].str.replace(r'\d', '')
df

,word_phoneme,phonetic_alternative_phoneme
0,OW1 K,OW1 K
1,S AA1 R OW0,S AA1 R OW0
2,K IH1 L AH0 G R AE2 M Z,K IH2 L HH AH0 G R AA1 M AH0 D IY0
3,T AE1 K S IY0,T EH1 K S IY0 IH0 S
4,B EH1 TH,B EH1 TH
...,...,...
7099,F AO1 R M AE2 T S,F AO1 R M EY2 T S
7100,AH0 B AW1 T,AA1 B AA0 T
7101,HH Y UW1 M AH0 D,HH AY1 W M AH0 N D
7102,F AO1 R,F AO1 ER0


In [ ]:
# Remove rows where columns are the same
df = df[df['word_phoneme'] != df['phonetic_alternative_phoneme']]

# Remove rows where columns only differ by the number of spaces
df = df[df.apply(lambda row: row['word_phoneme'].strip() != row['phonetic_alternative_phoneme'].strip(), axis=1)]

df.reset_index(drop=True, inplace=True)
print(df[['word_phoneme', 'phonetic_alternative_phoneme']])

                 word_phoneme        phonetic_alternative_phoneme
0     K IH1 L AH0 G R AE2 M Z  K IH2 L HH AH0 G R AA1 M AH0 D IY0
1               T AE1 K S IY0                 T EH1 K S IY0 IH0 S
2        D IH0 V IH1 ZH AH0 N           D IH1 V HH IH0 Z HH AA2 N
3      P AH0 T EH1 N SH AH0 L              P AA1 T AH0 N SH AH0 N
4       OW0 L IH1 M P IH0 K S               OW1 L IH0 F M AY2 K S
...                       ...                                 ...
5480        F AO1 R M AE2 T S                   F AO1 R M EY2 T S
5481              AH0 B AW1 T                         AA1 B AA0 T
5482         HH Y UW1 M AH0 D                  HH AY1 W M AH0 N D
5483                  F AO1 R                           F AO1 ER0
5484                    AH1 S                               AA1 Z

[5485 rows x 2 columns]


# train test split

In [ ]:
# Split the DataFrame into training and testing sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
print(train_df)
print(test_df)

                   word_phoneme     phonetic_alternative_phoneme
1620              L IH1 T AH0 L                    L AY1 T AH0 L
3000                L AH1 V ER0                      L AE1 V ER0
2347  S AY2 AH0 N T IH1 F IH0 K  S AY2 AH0 N T EH1 F IH0 K IH0 N
4969           F R AE1 NG K AH0               F R EH1 NG G AA0 K
5267                 B EY1 DH D                       B AY1 D TH
...                         ...                              ...
3772          S AH1 M TH IH0 NG                S AA1 M TH IH0 NG
5191           HH OW1 M W ER2 K                 HH AW1 M W ER2 K
5226            G L OW1 B AH0 L                    G L OW1 B AA2
5390                  P AE1 S T                        P EH1 S T
860           S AH0 M T AY1 M Z              S AA1 M T IY0 AH0 Z

[4388 rows x 2 columns]
              word_phoneme phonetic_alternative_phoneme
1468               B UH1 K                      B AH1 K
33     D IH0 S P L IY1 Z D        D IH1 S P L AH0 F S T
8      K W AY1 AH0 T L IY0 

In [ ]:
import pandas as pd

def remove_numbers(df):
    df_copy = df.copy()
    # Apply the function to remove numbers from both columns
    df_copy['word_phoneme'] = df_copy['word_phoneme'].apply(lambda x: ''.join([c for c in x if not c.isdigit()]))
    df_copy['phonetic_alternative_phoneme'] = df_copy['phonetic_alternative_phoneme'].apply(lambda x: ''.join([c for c in x if not c.isdigit()]))
    return df_copy

# Example usage
train_df_no_numbers = remove_numbers(train_df)
test_df_no_numbers = remove_numbers(test_df)

# Print the DataFrames without numbers
print("Train DataFrame without numbers:")
print(train_df_no_numbers)

print("\nTest DataFrame without numbers:")
print(test_df_no_numbers)

Train DataFrame without numbers:
               word_phoneme phonetic_alternative_phoneme
1620            L IH T AH L                  L AY T AH L
3000              L AH V ER                    L AE V ER
2347  S AY AH N T IH F IH K   S AY AH N T EH F IH K IH N
4969         F R AE NG K AH             F R EH NG G AA K
5267              B EY DH D                    B AY D TH
...                     ...                          ...
3772        S AH M TH IH NG              S AA M TH IH NG
5191         HH OW M W ER K               HH AW M W ER K
5226          G L OW B AH L                  G L OW B AA
5390               P AE S T                     P EH S T
860         S AH M T AY M Z             S AA M T IY AH Z

[4388 rows x 2 columns]

Test DataFrame without numbers:
           word_phoneme phonetic_alternative_phoneme
1468             B UH K                       B AH K
33    D IH S P L IY Z D          D IH S P L AH F S T
8      K W AY AH T L IY          K W EY AH T AO L IY
3752    K AH 

# save to csv

In [ ]:
# Save the training and testing sets to new CSV files
train_df_no_numbers.to_csv('data/train_word_output.csv', index=False)
test_df_no_numbers.to_csv('data/test_word_output.csv', index=False)

In [ ]:
import pandas as pd

def convert_csv_to_json(csv_path, json_output_path):
    df = pd.read_csv(csv_path)
    json_data = df.to_json(orient='records')

    with open(json_output_path, 'w') as json_file:
        json_file.write(json_data)

csv_file_path = 'data/train_word_output.csv'
json_output_file_path = 'data/train_word_output.json'
convert_csv_to_json(csv_file_path, json_output_file_path)

csv_file_path = 'data/test_word_output.csv'
json_output_file_path = 'data/test_word_output.json'
convert_csv_to_json(csv_file_path, json_output_file_path)

In [ ]:
# Load the dataset from the JSON files
dataset = load_dataset('json', data_files={'train': 'data/train_word_output.json',
                                           'test': 'data/test_word_output.json'}
                                           )

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [ ]:
print(dataset)
pprint(dataset['train'][:5])
pprint(dataset['test'][:5])

DatasetDict({
    train: Dataset({
        features: ['phonetic_alternative_phoneme', 'word_phoneme'],
        num_rows: 4388
    })
    test: Dataset({
        features: ['phonetic_alternative_phoneme', 'word_phoneme'],
        num_rows: 1097
    })
})
{'phonetic_alternative_phoneme': ['L AY T AH L',
                                  'L AE V ER',
                                  'S AY AH N T EH F IH K IH N',
                                  'F R EH NG G AA K',
                                  'B AY D TH'],
 'word_phoneme': ['L IH T AH L',
                  'L AH V ER',
                  'S AY AH N T IH F IH K',
                  'F R AE NG K AH',
                  'B EY DH D']}
{'phonetic_alternative_phoneme': ['B AH K',
                                  'D IH S P L AH F S T',
                                  'K W EY AH T AO L IY',
                                  'K AA M P L AH T',
                                  'S IH B L IH SH IH NG'],
 'word_phoneme': ['B UH K',
           

In [ ]:
# Split the dataset into train and validation sets
train_dataset, val_dataset = train_test_split(dataset['train'], test_size=0.1, random_state=42)

In [ ]:
from datasets import Dataset

train_data = Dataset.from_dict(train_dataset)  # Convert the train data to a dataset
val_data = Dataset.from_dict(val_dataset)      # Convert the validation data to a dataset

In [ ]:
print(type(train_data))
print(type(val_data))

<class 'datasets.arrow_dataset.Dataset'>
<class 'datasets.arrow_dataset.Dataset'>


In [ ]:
# Check column names in train dataset
print(train_data.column_names)

# Check column names in validation dataset
print(val_data.column_names)

['phonetic_alternative_phoneme', 'word_phoneme']
['phonetic_alternative_phoneme', 'word_phoneme']


In [ ]:
# Define the source and target language columns
source_lang = 'phonetic_alternative_phoneme'
target_lang = 'word_phoneme'
print(source_lang)

phonetic_alternative_phoneme


In [ ]:
# Define the max_length for padding and truncation
max_length = 512

In [ ]:
# Initialize the tokenizer
tokenizer = BartTokenizerFast.from_pretrained('facebook/bart-base')

# Tokenize the data
# The preprocess_function function is defined to preprocess the data by tokenizing the inputs and labels
def preprocess_function(examples):
    inputs = [f'{source_lang}: {text}' for text in examples[source_lang]]
    targets = examples[target_lang]
    encoding = tokenizer(inputs, padding=True, truncation=True, return_tensors='pt', max_length=max_length)
    model_inputs = {
        'input_ids': encoding['input_ids'].squeeze(),
        'attention_mask': encoding['attention_mask'].squeeze(),
        'labels': tokenizer(targets, padding=True, truncation=True, return_tensors='pt')['input_ids'].squeeze()
    }
    return model_inputs

In [ ]:
# Select a random data point from the train dataset
sample_data = train_data[0]

# Call the preprocess function on the sample data
processed_data = preprocess_function(sample_data)

# Inspect the output
print(processed_data)

{'input_ids': tensor([[    0,  3792,   261, 15557,  1215, 34268,  3693,  1215,  3792,   261,
         17185,    35,   274,     2,     1],
        [    0,  3792,   261, 15557,  1215, 34268,  3693,  1215,  3792,   261,
         17185,    35,  1437,  1437,     2],
        [    0,  3792,   261, 15557,  1215, 34268,  3693,  1215,  3792,   261,
         17185,    35,   226,     2,     1],
        [    0,  3792,   261, 15557,  1215, 34268,  3693,  1215,  3792,   261,
         17185,    35,  1437,  1437,     2],
        [    0,  3792,   261, 15557,  1215, 34268,  3693,  1215,  3792,   261,
         17185,    35,    83,     2,     1],
        [    0,  3792,   261, 15557,  1215, 34268,  3693,  1215,  3792,   261,
         17185,    35,   854,     2,     1],
        [    0,  3792,   261, 15557,  1215, 34268,  3693,  1215,  3792,   261,
         17185,    35,   112,     2,     1],
        [    0,  3792,   261, 15557,  1215, 34268,  3693,  1215,  3792,   261,
         17185,    35,  1437,  1437,   

In [ ]:
# Apply preprocess_function to train_data and val_data
train_data = train_data.map(preprocess_function, batched=True)
val_data = val_data.map(preprocess_function, batched=True)

Map:   0%|          | 0/3949 [00:00<?, ? examples/s]

Map:   0%|          | 0/439 [00:00<?, ? examples/s]

In [ ]:
# Prepare DataLoader for training and validation
train_dataloader = torch.utils.data.DataLoader(train_data, batch_size=16, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(val_data, batch_size=16)

In [ ]:
# define a data_collator function for batch processing
def data_collator(features):
    batch = {}
    # Pad input_ids and attention_mask to the maximum length within the batch
    max_length = max(len(feature['input_ids']) for feature in features)
    batch['input_ids'] = torch.stack([torch.tensor(feature['input_ids'] + [tokenizer.pad_token_id] * (max_length - len(feature['input_ids']))) for feature in features])
    batch['attention_mask'] = torch.stack([torch.tensor(feature['attention_mask'] + [0] * (max_length - len(feature['attention_mask']))) for feature in features])
    batch['labels'] = torch.stack([torch.tensor(feature['labels'] + [-100] * (max_length - len(feature['labels']))) for feature in features])
    return batch

# restart runtime to make sure accelerate>=0.20.1

In [ ]:
# ! pip install -U accelerate
# ! pip install -U transformers
import accelerate

accelerate.__version__

'0.25.0'

In [ ]:
# Define training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="spell_correction_tatoeba_word_level",
    evaluation_strategy="epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=30,
    predict_with_generate=True,
    push_to_hub=True,
)

In [ ]:
# verify passing the correct inputs to the trainer
print("Train Dataset:", train_data)
print("Validation Dataset:", val_data)
print("Tokenizer:", tokenizer)
print("Training Arguments:", training_args)

Train Dataset: Dataset({
    features: ['phonetic_alternative_phoneme', 'word_phoneme', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 3949
})
Validation Dataset: Dataset({
    features: ['phonetic_alternative_phoneme', 'word_phoneme', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 439
})
Tokenizer: BartTokenizerFast(name_or_path='facebook/bart-base', vocab_size=50265, model_max_length=1024, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': '<mask>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	1: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=

In [35]:
torch.cuda.empty_cache()


In [36]:
# model is initialized with the BARTForConditionalGeneration class and moved to the GPU if available.
model = BartForConditionalGeneration.from_pretrained('facebook/bart-base')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(50265, 768, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(50265, 768, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,), eps=

In [38]:
# The Seq2SeqTrainer is created with the defined model, training arguments, datasets, tokenizer, and data_collator
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=val_data,
    tokenizer=tokenizer,
    data_collator=data_collator
)

trainer.train()

  0%|          | 0/9900 [00:00<?, ?it/s]

/home/jinda/anaconda3/envs/cs224n/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


  0%|          | 0/37 [00:00<?, ?it/s]

{'eval_loss': 0.4191184639930725, 'eval_runtime': 5.1827, 'eval_samples_per_second': 84.704, 'eval_steps_per_second': 7.139, 'epoch': 1.0}
{'loss': 1.8152, 'learning_rate': 9.494949494949497e-06, 'epoch': 1.52}


/home/jinda/anaconda3/envs/cs224n/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


...

  0%|          | 0/37 [00:00<?, ?it/s]

{'eval_loss': 0.20506660640239716, 'eval_runtime': 5.1876, 'eval_samples_per_second': 84.624, 'eval_steps_per_second': 7.132, 'epoch': 29.0}


  0%|          | 0/37 [00:00<?, ?it/s]

{'eval_loss': 0.20489315688610077, 'eval_runtime': 5.1973, 'eval_samples_per_second': 84.467, 'eval_steps_per_second': 7.119, 'epoch': 30.0}
{'train_runtime': 3162.5468, 'train_samples_per_second': 37.46, 'train_steps_per_second': 3.13, 'train_loss': 0.25096462211223564, 'epoch': 30.0}


TrainOutput(global_step=9900, training_loss=0.25096462211223564, metrics={'train_runtime': 3162.5468, 'train_samples_per_second': 37.46, 'train_steps_per_second': 3.13, 'train_loss': 0.25096462211223564, 'epoch': 30.0})

In [39]:
# Load the trained model from Hugging Face
model = BartForConditionalGeneration.from_pretrained("matrixcc/spell_correction_tatoeba_word_level")

config.json:   0%|          | 0.00/1.74k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

In [40]:
# Move the model to the GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
model.eval()

BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(50265, 768, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(50265, 768, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,), eps=

In [94]:
test_dataset = dataset['test']
test_dataset['phonetic_alternative_phoneme']

['B AH K',
 'D IH S P L AH F S T',
 'K W EY AH T AO L IY',
 'K AA M P L AH T',
 'S IH B L IH SH IH NG',
 'F L AE SH',
 'T W AY Z',
 'AH M IH L IY AH',
 'R IH M HH OW T',
 'K OW HH AO R F AH Z AH N D Z',
 'D IH S K HH AA R D AH D',
 'L EH K ER',
 'K EH M P IH N Z',
 'AO ER Z',
 'S EH D',
 'R AY K W EY ER',
 'EH D V ER T AY Z IH NG',
 'R IH P HH ER Z',
 'D AA N AH HH AE L D',
 'S AO ER Z',
 'K AA T S',
 'G R AY D',
 'S AA F ER D',
 'T AW T AH L',
 'EY L W EH Z',
 'S P AY K IH NG',
 'S EH T ER IH D IY Z',
 'IH K S T ER N HH AH L IY',
 'B EH L D S',
 'S AY M IY TH AE L Y AH N AH',
 'EH P T',
 'S T R AE M IH NG',
 'F EH K ER T IY Z',
 'K AH N S IH L IH M AH N T',
 'S EY N R OW T IH NG',
 'R AY L OW D AY K T',
 'B UW T W IH D AE M',
 'HH OW P L AH Z',
 'IH Z AH B EH L AH',
 'K AA M IH SH IY N D AH N Z',
 'D EY N Y UW Z',
 'EH M B IY AH G R AH HH AH S',
 'S K EY S K R AY P ER',
 'IH NG K S P EH K T',
 'S AA T AH K R AH HH AH L',
 'T EY B L AH T S',
 'EH R K R EH F T',
 'IH M P AE L F IH S IH 

In [42]:
print(len(test_dataset))

1097


In [43]:
# Inspect column names
column_names = test_dataset.column_names
print("Column names:", column_names)

# Inspect column data types
for column in column_names:
    column_data = test_dataset[column]
    data_type = type(column_data[0]) if len(column_data) > 0 else "Unknown"
    print(f"Column name: {column}, Data type: {data_type}")

Column names: ['phonetic_alternative_phoneme', 'word_phoneme']
Column name: phonetic_alternative_phoneme, Data type: <class 'str'>
Column name: word_phoneme, Data type: <class 'str'>


In [44]:
# verification
# Test the model on speaker dataset
verify_test_dataset = dataset['test']
print(len(verify_test_dataset))
print((verify_test_dataset))

1097
Dataset({
    features: ['phonetic_alternative_phoneme', 'word_phoneme'],
    num_rows: 1097
})


In [95]:
dataset['test']['phonetic_alternative_phoneme']

['B AH K',
 'D IH S P L AH F S T',
 'K W EY AH T AO L IY',
 'K AA M P L AH T',
 'S IH B L IH SH IH NG',
 'F L AE SH',
 'T W AY Z',
 'AH M IH L IY AH',
 'R IH M HH OW T',
 'K OW HH AO R F AH Z AH N D Z',
 'D IH S K HH AA R D AH D',
 'L EH K ER',
 'K EH M P IH N Z',
 'AO ER Z',
 'S EH D',
 'R AY K W EY ER',
 'EH D V ER T AY Z IH NG',
 'R IH P HH ER Z',
 'D AA N AH HH AE L D',
 'S AO ER Z',
 'K AA T S',
 'G R AY D',
 'S AA F ER D',
 'T AW T AH L',
 'EY L W EH Z',
 'S P AY K IH NG',
 'S EH T ER IH D IY Z',
 'IH K S T ER N HH AH L IY',
 'B EH L D S',
 'S AY M IY TH AE L Y AH N AH',
 'EH P T',
 'S T R AE M IH NG',
 'F EH K ER T IY Z',
 'K AH N S IH L IH M AH N T',
 'S EY N R OW T IH NG',
 'R AY L OW D AY K T',
 'B UW T W IH D AE M',
 'HH OW P L AH Z',
 'IH Z AH B EH L AH',
 'K AA M IH SH IY N D AH N Z',
 'D EY N Y UW Z',
 'EH M B IY AH G R AH HH AH S',
 'S K EY S K R AY P ER',
 'IH NG K S P EH K T',
 'S AA T AH K R AH HH AH L',
 'T EY B L AH T S',
 'EH R K R EH F T',
 'IH M P AE L F IH S IH 

In [96]:
test_data = test_dataset.map(preprocess_function, batched=True, batch_size=len(test_dataset))
model.eval()

Map:   0%|          | 0/1097 [00:00<?, ? examples/s]

BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(50265, 768, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(50265, 768, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,), eps=

In [97]:
predictions = []
references = []

for example in test_dataset:
    # This creates an input text by combining the values of the 'speaker' and 'prediction' fields from the current example.
    # It assumes that the example is a dictionary-like object with keys 'speaker' and 'prediction'.
    input_text = f"{example['phonetic_alternative_phoneme']}"

    # This creates a context where no gradients are computed, which can improve efficiency during inference.
    with torch.no_grad():

        # This tokenizes the input_text using the tokenizer and converts it into input IDs as a PyTorch tensor.
        input_ids = tokenizer.encode(input_text, return_tensors='pt').to(device)

        # The generate method generates the output sequence based on the provided input.
        outputs = model.generate(input_ids=input_ids, max_length=max_length)

    predicted_sentence = tokenizer.decode(outputs[0], skip_special_tokens=True)

    references.append(example['word_phoneme'])
    predictions.append(predicted_sentence)

In [98]:
# Verify that the number of predictions and references are the same
if len(predictions) == len(references):
    print("Number of predictions and references are the same.")
else:
    print("Mismatch in the number of predictions and references.")

Number of predictions and references are the same.


In [99]:
# Print the number of predictions and references
print("Number of predictions:", len(predictions))
print("Number of references:", len(references))

# print the length of the dataset
print("Number of rows in dataset:", len(test_dataset))

Number of predictions: 1097
Number of references: 1097
Number of rows in dataset: 1097


In [100]:
for i in range(min(len(test_dataset['phonetic_alternative_phoneme']), len(predictions), len(references))):
    print(f"Sample {i+1}:")
    print("Test Dataset:", test_dataset['phonetic_alternative_phoneme'][i])
    print("Prediction:", predictions[i])
    print("Reference:", references[i])
    print("\n")

Sample 1:
Test Dataset: B AH K
Prediction: B UH K
Reference: B UH K


Sample 2:
Test Dataset: D IH S P L AH F S T
Prediction: D IH S P L EH S T
Reference: D IH S P L IY Z D


Sample 3:
Test Dataset: K W EY AH T AO L IY
Prediction: K W AY AH T L IY
Reference: K W AY AH T L IY


Sample 4:
Test Dataset: K AA M P L AH T
Prediction: K AH M P L EY T
Reference: K AH M P L IY T


Sample 5:
Test Dataset: S IH B L IH SH IH NG
Prediction: S IH B L IY S IH NG
Reference: S IH B L IH NG Z


Sample 6:
Test Dataset: F L AE SH
Prediction: F L AH SH
Reference: F L AH SH


Sample 7:
Test Dataset: T W AY Z
Prediction: T W IY Z
Reference: T W AY S


Sample 8:
Test Dataset: AH M IH L IY AH
Prediction: AH M IY L IY
Reference: AH M IY L IY AH


Sample 9:
Test Dataset: R IH M HH OW T
Prediction: R IH M OW T
Reference: R IH M OW T


Sample 10:
Test Dataset: K OW HH AO R F AH Z AH N D Z
Prediction: K AO R F AH N D Z
Reference: K AO R AH S P AA N D Z


Sample 11:
Test Dataset: D IH S K HH AA R D AH D
Prediction: 

In [101]:
from jiwer import wer, cer

# Assuming 'predictions' and 'references' are your sequences
# Calculate Word Error Rate (WER)
wer_value = wer(predictions, references)
wer_percentage = wer_value * 100
print(f"WER: {wer_percentage:.2f}%")

# Calculate Character Error Rate (CER)
cer_value = cer(predictions, references)
cer_percentage = cer_value * 100
print(f"CER: {cer_percentage:.2f}%")

WER: 18.45%
CER: 13.03%
